### Introduction to the data

In [1]:
import sqlite3 as sql
conn = sql.connect("nominations.db")

show_schema = "pragma table_info('nominations')"
schema = conn.execute(show_schema).fetchall()
#print(schema)

for item in schema:
    print(item)

q1 = "select * from nominations limit 10"
first_ten = conn.execute(q1).fetchall()
#print(first_ten)

for item in first_ten:
    print(item)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

### Creating the ceremonies table

In [2]:
create_table = '''
create table ceremonies (id integer primary key, Year integer, Host text)
'''
conn.execute(create_table)

years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal")]

# insert into table using executemany plus placeholder (?)
insert = '''
insert into ceremonies (Year, Host) values (?,?)
'''
conn.executemany(insert, years_hosts)
q2 = "select * from ceremonies"
print(conn.execute(q2).fetchmany(10))

table_name = "ceremonies"
conn.execute(show_schema).fetchall()

[(1, 2010, 'Steve Martin'), (2, 2009, 'Hugh Jackman'), (3, 2008, 'Jon Stewart'), (4, 2007, 'Ellen DeGeneres'), (5, 2006, 'Jon Stewart'), (6, 2005, 'Chris Rock'), (7, 2004, 'Billy Crystal'), (8, 2003, 'Steve Martin'), (9, 2002, 'Whoopi Goldberg'), (10, 2001, 'Steve Martin')]


[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

### Foreign Key constraint

Since we'll be creating relations using foreign keys, we need to turn on foreign key constraints.

In [3]:
enable_fk = "PRAGMA foreign_keys = ON;"
conn.execute(enable_fk)

### Setting up the relations

In [4]:
# creating the table
create_nom_two = '''
create table nominations_two (
    id integer primary key, 
    category text, nominee text, movie text, 
    character text, won integer, ceremony_id integer,
    foreign key(ceremony_id) references ceremonies(id))
'''
conn.execute(create_nom_two)

# extracting the values
select_all_nom = '''
select n.category, n.nominee, n.movie, 
n.character, n.won, ceremonies.id
from nominations as n
inner join ceremonies on n.year == ceremonies.year ;
'''
joined_nominations = conn.execute(select_all_nom).fetchall()

# populating the table
insert_nominations_two = '''
insert into nominations_two 
  (category, nominee, movie, character, won, ceremony_id) 
  values (?,?,?,?,?,?)
'''
conn.executemany(insert_nominations_two, joined_nominations)

verify = "select * from nominations_two limit 5"
conn.execute(verify).fetchall()

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1),
 (3,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  0,
  1),
 (4,
  'Actor -- Leading Role',
  'Colin Firth',
  "The King's Speech",
  'King George VI',
  1,
  1),
 (5,
  'Actor -- Leading Role',
  'James Franco',
  '127 Hours',
  'Aron Ralston',
  0,
  1)]

### Deleting and Renaming tables

In [5]:
drop_table = "drop table nominations"
conn.execute(drop_table)

alter_table = "alter table nominations_two rename to nominations"
conn.execute(alter_table)


### Creating a join table

In [6]:
create_movies = "create table movies (id integer primary key, movie text)"
create_actors = "create table actors (id integer primary key, actor text)"
create_movies_actors = "create table movies_actors (id integer primary key, movie_id integer references movies(id), actor_id integer references actors(id))"

conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movies_actors)

print(conn.execute("pragma table_info(movies)").fetchall())
print(conn.execute("pragma table_info(actors)").fetchall())
print(conn.execute("pragma table_info(movies_actors)").fetchall())

[(0, 'id', 'integer', 0, None, 1), (1, 'movie', 'text', 0, None, 0)]
[(0, 'id', 'integer', 0, None, 1), (1, 'actor', 'text', 0, None, 0)]
[(0, 'id', 'integer', 0, None, 1), (1, 'movie_id', 'integer', 0, None, 0), (2, 'actor_id', 'integer', 0, None, 0)]


### Populating the movies and actors tables

In [7]:
insert_movies = "insert into movies (movie) select distinct movie from nominations;"
conn.execute(insert_movies)

insert_actors = "insert into actors (actor) select distinct nominee from nominations;"
conn.execute(insert_actors)

In [8]:
print(conn.execute("select * from movies limit 5;").fetchall())
print(conn.execute("select * from actors limit 5;").fetchall())

[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


### Populating a join table

We can populate the join table using a inner join. In the query below we'd get a join table with movies that have the same order as in `nominations` table. 

In [26]:
inner_join = '''
select movies.id, actors.id, n.movie, n.nominee from movies 
inner join nominations n on movies.movie == n.movie
inner join actors on n.nominee = actors.actor
limit 10
'''
conn.execute(inner_join).fetchall()

[(1, 1, 'Biutiful', 'Javier Bardem'),
 (2, 19, 'True Grit', 'Hailee Steinfeld'),
 (2, 2, 'True Grit', 'Jeff Bridges'),
 (3, 3, 'The Social Network', 'Jesse Eisenberg'),
 (4, 4, "The King's Speech", 'Colin Firth'),
 (4, 10, "The King's Speech", 'Geoffrey Rush'),
 (4, 17, "The King's Speech", 'Helena Bonham Carter'),
 (5, 5, '127 Hours', 'James Franco'),
 (6, 16, 'The Fighter', 'Amy Adams'),
 (6, 6, 'The Fighter', 'Christian Bale')]

We can also populate the join table using the `executemany` method plus using placeholders. For the sake of the project, we'll use this second solution. Both are valid. 

In [28]:
placeholder_query = "select movie,nominee from nominations"
placeholder = conn.execute(placeholder_query).fetchall()

insert_query = '''
insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?),
                                                        (select id from actors where actor == ?));
'''
conn.executemany(insert_query, placeholder)

conn.execute("select * from movies_actors limit 10;").fetchall()

[(1, 1, 1),
 (2, 2, 2),
 (3, 3, 3),
 (4, 4, 4),
 (5, 5, 5),
 (6, 6, 6),
 (7, 7, 7),
 (8, 8, 8),
 (9, 9, 9),
 (10, 4, 10)]

In [ ]:
# finally release the resources
conn.close()